In [1]:
import pandas as pd
import csv
import time
import glob

In [2]:
data_path = "data/"

In [3]:
list_data = glob.glob(data_path+'*.csv')

In [4]:
detected_file = []
for event_file in list_data:
    detected_file.append(pd.read_csv(event_file, sep = "|", index_col=0))

### concatenate in one dataframe the data from 2010 to 2016

In [5]:
detected_event_data = pd.concat(detected_file)

In [6]:
detected_event_data.reset_index(inplace=True, drop=True)
detected_event_data.head()

,hashtag,dayOfTweet,usersPerHashtag,spamEvent,std,approxLocation
0,#tedxzh,2010-10-20,5,False,178.505886,"('47.383', '8.536')"
1,#gotthard,2010-10-15,4,False,159.250786,"('47.567', '7.597')"
2,#esc,2010-05-29,4,False,61.585952,"('47.162', '8.291')"
3,#fb,2010-10-07,3,False,257.948167,"('46.289', '7.972')"
4,#tedxgeneva,2010-12-06,2,False,153.596919,"('46.194', '6.154')"


##  PREPROCESSING 

### Keep only the false of the spamEvent

In [7]:
detected_event_data[detected_event_data.spamEvent == False]

,hashtag,dayOfTweet,usersPerHashtag,spamEvent,std,approxLocation
0,#tedxzh,2010-10-20,5,False,178.505886,"('47.383', '8.536')"
1,#gotthard,2010-10-15,4,False,159.250786,"('47.567', '7.597')"
2,#esc,2010-05-29,4,False,61.585952,"('47.162', '8.291')"
3,#fb,2010-10-07,3,False,257.948167,"('46.289', '7.972')"
4,#tedxgeneva,2010-12-06,2,False,153.596919,"('46.194', '6.154')"
5,#ff,2010-12-03,2,False,105.322863,"('47.588', '9.624')"
12,#brexit,2016-06-24,186,False,261.680985,"('47.719', '8.623')"
13,#euro2016,2016-06-27,96,False,165.867610,"('46.954', '7.395')"
14,#suifra,2016-06-19,95,False,108.932860,"('46.529', '6.652')"
15,#frapor,2016-07-10,87,False,174.980705,"('46.205', '6.143')"


### 

In [8]:
detected_event_data[detected_event_data.usersPerHashtag > 10].shape

(3698, 6)

In [9]:
detected_event_data = detected_event_data[detected_event_data.usersPerHashtag > 10]

####  rename the std column to avoid any trouble with series.std

In [10]:
detected_event_data.rename(columns={'std':'std_metrics'}, inplace=True)

#### change the date format and add in the same time the number of tweets per day

In [11]:
from datetime import datetime

In [12]:
date_format = "%Y-%m-%d"

In [13]:
detected_event_data['dayOfTweet'] = pd.to_datetime(detected_event_data['dayOfTweet'],format = date_format,errors='coerce')

In [14]:
detected_event_data.head()

,hashtag,dayOfTweet,usersPerHashtag,spamEvent,std_metrics,approxLocation
12,#brexit,2016-06-24,186,False,261.680985,"('47.719', '8.623')"
13,#euro2016,2016-06-27,96,False,165.867610,"('46.954', '7.395')"
14,#suifra,2016-06-19,95,False,108.932860,"('46.529', '6.652')"
15,#frapor,2016-07-10,87,False,174.980705,"('46.205', '6.143')"
16,#sui,2016-06-19,87,False,172.971667,"('46.894', '7.055')"


In [15]:
new_date_format = '%Y-%m-%dT%H:%M:%SZ'

In [16]:
def change_date_format(ds):
    return ds.strftime('%Y-%m-%dT%H:%M:%SZ')
    

In [17]:
detected_event_data['dayOfTweet'] = detected_event_data['dayOfTweet'].apply(change_date_format)

In [18]:
detected_event_data.head()

,hashtag,dayOfTweet,usersPerHashtag,spamEvent,std_metrics,approxLocation
12,#brexit,2016-06-24T00:00:00Z,186,False,261.680985,"('47.719', '8.623')"
13,#euro2016,2016-06-27T00:00:00Z,96,False,165.867610,"('46.954', '7.395')"
14,#suifra,2016-06-19T00:00:00Z,95,False,108.932860,"('46.529', '6.652')"
15,#frapor,2016-07-10T00:00:00Z,87,False,174.980705,"('46.205', '6.143')"
16,#sui,2016-06-19T00:00:00Z,87,False,172.971667,"('46.894', '7.055')"


In [34]:
detected_event_data.sort_values('usersPerHashtag',ascending=False,)

,hashtag,dayOfTweet,usersPerHashtag,spamEvent,std_metrics,eventLatitude,eventLongitude
12,#brexit,2016-06-24T00:00:00Z,186,False,261.680985,47.719,8.623
8465,#prayforparis,2015-11-14T00:00:00Z,131,False,343.310548,46.205,6.143
15295,#nma,2013-12-14T00:00:00Z,127,False,84.671164,47.262,6.032
8466,#jesuischarlie,2015-01-07T00:00:00Z,119,False,170.470379,47.485,6.914
8467,#charliehebdo,2015-01-07T00:00:00Z,113,False,210.028082,46.205,6.143
3667,#sui,2014-07-01T00:00:00Z,111,False,160.228891,46.236,6.080
15296,#nma,2013-01-26T00:00:00Z,104,False,82.037253,46.221,6.109
3668,#ger,2014-07-13T00:00:00Z,102,False,179.350587,47.299,8.445
3669,#frahon,2014-06-15T00:00:00Z,100,False,90.788794,47.528,6.817
13,#euro2016,2016-06-27T00:00:00Z,96,False,165.867610,46.954,7.395


In [19]:
import re

In [20]:
def getLatitude(df):
    coord_list = re.findall(r"[-+]?\d*\.\d+|\d+", df.approxLocation)
    return float(coord_list[0])

In [21]:
def getLongitude(df):
    coord_list = re.findall(r"[-+]?\d*\.\d+|\d+", df.approxLocation)
    return float(coord_list[1])

In [22]:
detected_event_data['eventLatitude'] = detected_event_data.apply(getLatitude,axis=1)

In [23]:
detected_event_data['eventLongitude'] = detected_event_data.apply(getLongitude,axis=1)

In [24]:
detected_event_data.drop('approxLocation',axis=1,inplace=True)

In [25]:
detected_event_data.head()

,hashtag,dayOfTweet,usersPerHashtag,spamEvent,std_metrics,eventLatitude,eventLongitude
12,#brexit,2016-06-24T00:00:00Z,186,False,261.680985,47.719,8.623
13,#euro2016,2016-06-27T00:00:00Z,96,False,165.867610,46.954,7.395
14,#suifra,2016-06-19T00:00:00Z,95,False,108.932860,46.529,6.652
15,#frapor,2016-07-10T00:00:00Z,87,False,174.980705,46.205,6.143
16,#sui,2016-06-19T00:00:00Z,87,False,172.971667,46.894,7.055


on the json we'll keep hashtag, dayOfTweet, usersPerHashtag, std_metrics,eventlatitude and eventlongitude

## finally we'll get a json file like that 

In [26]:
{"type":"Feature","geometry":{"type":"Point","coordinates":[47.383,8.536]},"geometry_name":"tweet_information","properties":{"hashtag":"#tedxzh","dayOfTweet":"2010-10-20T00:00:00Z","eventLongitude":8.536,"eventLatitude":47.383,"std_metrics":195.753539, "usersPerHashtag":8, "std_min":63.0, "std_max":202.0}}

{'geometry': {'coordinates': [47.383, 8.536], 'type': 'Point'},
 'geometry_name': 'tweet_information',
 'properties': {'dayOfTweet': '2010-10-20T00:00:00Z',
  'eventLatitude': 47.383,
  'eventLongitude': 8.536,
  'hashtag': '#tedxzh',
  'std_max': 202.0,
  'std_metrics': 195.753539,
  'std_min': 63.0,
  'usersPerHashtag': 8},
 'type': 'Feature'}

In [27]:
import json

In [28]:
# gjson is th emain dictionary
gjson_dict={}
gjson_dict["type"]= "FeatureCollection"
feat_list = []


In [29]:
gjson_dict

{'type': 'FeatureCollection'}

In [30]:
from geojson import Point

In [31]:

# Loop through all the courses, building a list entry which is itself a dictionary
# Each of these dictionaries has nested within it a type dictionary, which contains a point dictionary and a properties dictionary 
for index in detected_event_data.index:
    type_dict = {}
    pt_dict = {}
    prop_dict = {}
    type_dict["type"]= "Feature"
    pt_dict["type"]="Point"

    # GEOJSON looks for long,lat so reverse order
    type_dict["geometry"]=Point((detected_event_data.loc[index].eventLongitude,
                                 detected_event_data.loc[index].eventLatitude))
    
    prop_dict["dayOfTweet"] = detected_event_data.loc[index].dayOfTweet
    prop_dict["hashtag"] = detected_event_data.loc[index].hashtag
    prop_dict["std_metrics"]= detected_event_data.loc[index].std_metrics
    prop_dict["eventLongitude"]= detected_event_data.loc[index].eventLongitude
    prop_dict["eventLatitude"]= detected_event_data.loc[index].eventLatitude
    prop_dict["usersPerHashtag"] = detected_event_data.loc[index].usersPerHashtag
    prop_dict['min_std'] = round(detected_event_data.std_metrics.min())
    prop_dict['max_std'] = round(detected_event_data.std_metrics.max())
    type_dict["properties"]=prop_dict
    feat_list.append(type_dict)
    
gjson_dict["features"] = feat_list



In [32]:
gjson_dict

{'features': [{'geometry': {"coordinates": [8.6229999999999993, 47.719000000000001], "type": "Point"},
   'properties': {'dayOfTweet': '2016-06-24T00:00:00Z',
    'eventLatitude': 47.719000000000001,
    'eventLongitude': 8.6229999999999993,
    'hashtag': '#brexit',
    'max_std': 586.0,
    'min_std': 20.0,
    'std_metrics': 261.68098467996623,
    'usersPerHashtag': 186},
   'type': 'Feature'},
  {'geometry': {"coordinates": [7.3949999999999996, 46.954000000000001], "type": "Point"},
   'properties': {'dayOfTweet': '2016-06-27T00:00:00Z',
    'eventLatitude': 46.954000000000001,
    'eventLongitude': 7.3949999999999996,
    'hashtag': '#euro2016',
    'max_std': 586.0,
    'min_std': 20.0,
    'std_metrics': 165.86761039283476,
    'usersPerHashtag': 96},
   'type': 'Feature'},
  {'geometry': {"coordinates": [6.6520000000000001, 46.529000000000003], "type": "Point"},
   'properties': {'dayOfTweet': '2016-06-19T00:00:00Z',
    'eventLatitude': 46.529000000000003,
    'eventLongitude

In [33]:

# Serialize JSON for writing to file
json_path = './data/event_detection.json'
with open(json_path, 'w') as outfile:
     json.dump(gjson_dict, outfile, sort_keys = True, indent = 4, ensure_ascii=False)
